In [ ]:
# allState.shape: (238, 14, 599200)
# liftdrag_arr.shape: (238, 2, 599200/333) = (238, 2, 1799)

In [1]:
import os
import pickle
import datetime
import numpy as np
import pandas as pd

In [2]:
cur_dir = os.getcwd()
main_dir = os.path.dirname(os.path.dirname(cur_dir))
data_dir = os.path.join(main_dir, "ConsolidatedData", "Training1_Jan2023")

all_df_dataname = "videotimed_labeled_consolidated_all.pkl"

all_df_pkl = os.path.join(data_dir, all_df_dataname)

with open(all_df_pkl, 'rb') as f:
  all_df = pickle.load(f)

In [3]:
run_all_datetimes = all_df["Date/Time"].apply(lambda x: datetime.datetime.strptime(x, "%Y-%m-%d_%H-%M-%S-%f"))

unique_labels = all_df["Label"].unique()
unique_labels = np.delete(unique_labels, 0)

In [ ]:
run_timing = dict()

for label in unique_labels:
  print (f"Processing run: {label}")

  run_start_row_eds = all_df[all_df["Label"] == label].index[0]
  run_start_time = run_all_datetimes.iloc[run_start_row_eds]
  run_row_cnt_eds = all_df["Label"].value_counts()[label]
  if run_start_row_eds + run_row_cnt_eds < run_all_datetimes.shape[0]:
    run_end_time = run_all_datetimes.iloc[run_start_row_eds + run_row_cnt_eds]
  else:
    run_end_time = run_all_datetimes.iloc[-1]
  
  run_timing[label] = (run_start_time, run_end_time)

In [5]:
shortest = run_timing[unique_labels[0]][1] - run_timing[unique_labels[0]][0]
for label,timing in run_timing.items():
  duration = timing[1] - timing[0]
  if duration < shortest:
    shortest = duration

In [6]:
DAQ_SAMPLERATE = 10000
WINDOW_SIZE = 333
liftdrag_arr = np.zeros((len(unique_labels), 2, int((shortest.seconds + shortest.microseconds/1000000) * DAQ_SAMPLERATE // WINDOW_SIZE)))

timestep = datetime.timedelta(0,0,WINDOW_SIZE/DAQ_SAMPLERATE * 1000000) #Predicitons in every 333 SG/PZT samples. In time-scale, we need to make predictions in every 333/10000 second.

In [ ]:
for cnt,label in enumerate(unique_labels):
  print (f"Processing run {label}")
  
  (run_start_time, run_end_time) = run_timing[label]
  t = run_start_time
  start_row_ix = run_all_datetimes[run_all_datetimes == t].index[0]
  row_ix = start_row_ix
  
  sample_id = 0
  while t < run_end_time:
    while run_all_datetimes.iloc[row_ix] <= t+timestep:
      row_ix += 1
    
    wt_lift = all_df["Lift (lbf)"].iloc[start_row_ix:row_ix].mean()
    wt_drag = all_df["Drag (lbf)"].iloc[start_row_ix:row_ix].mean()

    start_row_ix = row_ix
    t += timestep

    if sample_id + 1 <= liftdrag_arr.shape[2]:
      liftdrag_arr[cnt, 0, sample_id] = wt_lift
      liftdrag_arr[cnt, 1, sample_id] = wt_drag
    
    sample_id += 1

In [9]:
liftdrag_arr.shape

(238, 2, 1799)

In [11]:
output_folder = os.path.join(main_dir, "KerasML", "Training1_Jan2023", "data")
output_file = os.path.join(output_folder, "sttr_aug2022_liftdraglabels.npy")

np.save(output_file,liftdrag_arr)
